# Crime Dataset
In this section we are going to analyze the Crime dataset. 
This large dataset contains 128 columns, which need to be filtered in order to extract important features for our analysis: in particular, we are interested in the columns:
- state
- the population (in order to weight the values according to the population)
- NumUnderPov (the number of people under the poverty level)
- PctPopUnderPov (percentage of people under the poverty level)
- PctLess9thGrade (percentage with a level of instruction less than a 9th grade)
- PctNotHSGrad (percentage of people without high school diploma)
- PctBSorMore (percentage of people with bachelor or more)
- PctUnemployed (percentage of people unemployed)
- PctNotSpeakEnglWell (People not speaking english well) 
- MedNumBR (median number of bedrooms per house)
- PctWOFullPlumb (percentage of houses without full plumbing facilities),
- MedRent (medium rent)
- MedRentPctHousInc (median rent as percentage of income)
- NumInShelters (homeless people in shelters)
- NumStreet (homeless people in the street)
- LemasSwornFT (number of full time police officers)
- LemasSwFTPerPop (police officers per 100k inhabitants)
- LemasTotalReq (total requests for police)
- LemasTotReqPerPop (requests per police per 100K inhabitants)
- PolicPerPop (police officers per 100k inhabitants) difference with above?
- NumKindsDrugsSeiz (Number of different kinds of drugs sized)
- PctUsePubTrans (percentage of people using public means of transport for commuting)
- ViolentCrimesPerPop (Number of total crimes per 100k population)

In [1]:
from requests import get
import pandas as pd
import numpy as np
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup